In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

# Set up driver
driver = webdriver.Chrome()

# Open website
driver.get("https://www.coromandel.biz/crop-schedules/")

# Wait for page to load
time.sleep(5)

# Select each state
states_dropdown = Select(driver.find_element(By.ID, "stateOnchange"))
states = [option.text for option in states_dropdown.options][1:]  # Skip first option ("--Select--")

# Create an empty list to store dataframes
dfs = []

for state in states:
    states_dropdown.select_by_visible_text(state)
    time.sleep(2)

    # Select each crop
    crops_dropdown = Select(driver.find_element(By.ID, "cropname_wrapper"))
    crops = [option.text for option in crops_dropdown.options][1:]  # Skip first option ("--Select--")
    for crop in crops:
        crops_dropdown.select_by_visible_text(crop)
        time.sleep(2)

        # Click search button
        button = driver.find_element(By.XPATH, '//*[@id="searchCropSchedular"]/div/div[4]/button')
        button.click()
        time.sleep(5)

        # Scrape data
        table = driver.find_element(By.ID, "crop_table_wrapper")
        rows = table.find_elements(By.TAG_NAME, "tr")
        data = []
        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 2:
                state = cells[0].text
                crop = cells[1].text
                for i in range(2, len(cells), 2):
                    try:
                        month = cells[i].text
                        dosage = cells[i+1].text
                        data.append({"State": state, "Crop": crop, "Month": month, "Dosage": dosage})
                    except IndexError:
                        continue
        
        # Add list as a dataframe to the list of dataframes
        df = pd.DataFrame(data)
        dfs.append(df)

# Concatenate dataframes in the list into a single dataframe
df = pd.concat(dfs, ignore_index=True)

# Save data to CSV file
df.to_csv('crop_data.csv', index=False)

# Close driver
driver.quit()

# Print scraped data
print(df)


     State                         Crop                   Month Dosage
0     0-10                        Basal                  BOSMAX   Soil
1                                           GroPlus/ Gromor SSP   Soil
2                                                    Gromor MOP   Soil
3       30             Button Formation  Gromor Calcium nitrate   Soil
4                                                   Gromor Urea   Soil
...    ...                          ...                     ...    ...
4391     1                    Flowering                   NRICH   Soil
4392   0.5                                    Gromor 15-15-15-9   Soil
4393  0.25                               Gromor Calcium Nitrate   Soil
4394     1  Fruit Formation/Development       Gromor 15-15-15-9   Soil
4395                                                      KRICH   Soil

[4396 rows x 4 columns]
